# RDF-centered conversion
This notebook will convert ZIB triples to capacity fields.

In [1]:
from rdflib import Graph, URIRef, Namespace
from rdflib.namespace import RDFS,SKOS, RDF
from pathlib import Path

## ZIB ontology
A small sample ontology to be able to describe ZIB concepts.

This ontology contains the following ZIB classes.

| Name          | Description  |
|---------------|--------------|
| [MedicationUse](https://zibs.nl/wiki/MedicationUse2-v1.1.1(2020EN)) |    |
| [PharmaceuticalProduct](https://zibs.nl/wiki/PharmaceuticalProduct-v2.1.2(2020EN)) | Partial information model used in MedicationUse |



In [2]:
ZIB = Namespace('http://example.org/ZIB/')
zib_ontology = '../../ZIB/zib.owl'
zib_ontology = Graph(identifier=ZIB).parse(zib_ontology)

print(zib_ontology.serialize(format='turtle').decode())

@prefix : <http://example.org/ZIB#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

<http://example.org/ZIB> a owl:Ontology .

:MedicationUse a owl:Class ;
    rdfs:subClassOf :ZibConcept ;
    owl:disjointWith :PharmaceuticalProduct .

:hasZibRecord a owl:FunctionalProperty,
        owl:InverseFunctionalProperty,
        owl:ObjectProperty ;
    rdfs:subPropertyOf :zibProperty .

:medicationCode a owl:FunctionalProperty,
        owl:InverseFunctionalProperty,
        owl:ObjectProperty ;
    rdfs:subPropertyOf :zibProperty .

:PharmaceuticalProduct a owl:Class ;
    rdfs:subClassOf :ZibConcept .

:ZibConcept a owl:Class .

:zibProperty a owl:ObjectProperty ;
    rdfs:subPropertyOf owl:topObjectProperty .




# Sample records
Using the ZIB ontology some sample records are created for patient Bob which uses a medication called *quinidine* which has *C01BA01* within the ATC codesystem.

In [3]:
# Namespaces
ZIB = Namespace('http://example.org/ZIB/')
ATC = Namespace('http://purl.bioontology.org/ontology/ATC/')
UATC = Namespace('http://purl.bioontology.org/ontology/UATC/')


patient = Namespace('http://example.org/patient/')
zib_record = Namespace('http://example.org/zib_record/')

# Sample medicationUse zib
patient_graph = Graph()
patient_graph.bind('ZIB', ZIB)
patient_graph.bind('ATC', ATC)
patient_graph.bind('UATC', UATC)
patient_graph.bind('RDFS', RDFS)
patient_graph.bind('SKOS', SKOS)


patient_graph.add((patient.bob, ZIB.hasZibRecord, zib_record.bobsmedication))
patient_graph.add((zib_record.bobsmedication, RDF.type, ZIB.MedicationUse))
patient_graph.add((zib_record.bobsmedication, ZIB.medicationCode, UATC.C01BA01))

print(patient_graph.serialize(format='turtle').decode())

@prefix UATC: <http://purl.bioontology.org/ontology/UATC/> .
@prefix ZIB: <http://example.org/ZIB/> .

<http://example.org/patient/bob> ZIB:hasZibRecord <http://example.org/zib_record/bobsmedication> .

<http://example.org/zib_record/bobsmedication> a ZIB:MedicationUse ;
    ZIB:medicationCode UATC:C01BA01 .




# Adding ATC ontology


In [4]:
from zib_uploader.tools import get

atc_path = 'atc.ttl'
atc_url = 'http://data.bioontology.org/ontologies/ATC/submissions/12/download?apikey=8b5b7825-538d-40e0-9e9e-5ab9274a9aeb'

atc_file = get(atc_url)

patient_graph.parse(str(atc_file), format='turtle')

<Graph identifier=N3fcb51c8cf3b4f0cb6fe9e0c17603659 (<class 'rdflib.graph.Graph'>)>

Now we need to check whether we can use the ATC ontology to infer additional information about Mr. Bobs medication use.

A *DeductiveClosure* is an expansion of the knowledge base with all knowledge that can logically be derived from the original knowledge.

In [5]:
# import owlrl

# owlrl.DeductiveClosure(owlrl.CombinedClosure.RDFS_OWLRL_Semantics).expand(patient_graph)

It actually turns out that transitive relations can also be deduced without a reasoning engine by adding a '\*' to the transitive property.

If this is the most complicated reasoning we need we do not need a reasoning engine.

In [7]:
query = \
'''
select ?medication ?superclass
where {    
   <http://example.org/patient/bob> ZIB:hasZibRecord ?zibRecord .
    ?zibRecord ZIB:medicationCode ?medication .
    ?medication RDFS:subClassOf* ?superclass
}
'''

result = patient_graph.query(query)

for r in result:
    print(f'Medication {r[0]} is subclass of {r[1]}')

Medication http://purl.bioontology.org/ontology/UATC/C01BA01 is subclass of http://purl.bioontology.org/ontology/UATC/C01BA01
Medication http://purl.bioontology.org/ontology/UATC/C01BA01 is subclass of http://purl.bioontology.org/ontology/UATC/C01BA
Medication http://purl.bioontology.org/ontology/UATC/C01BA01 is subclass of http://purl.bioontology.org/ontology/UATC/C01B
Medication http://purl.bioontology.org/ontology/UATC/C01BA01 is subclass of http://purl.bioontology.org/ontology/UATC/C01
Medication http://purl.bioontology.org/ontology/UATC/C01BA01 is subclass of http://purl.bioontology.org/ontology/UATC/C
Medication http://purl.bioontology.org/ontology/UATC/C01BA01 is subclass of http://www.w3.org/2002/07/owl#Thing
